# Data loader experiments

In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data import *
from local.notebook.showdoc import show_doc

In [ ]:
import string
letters = list(string.ascii_lowercase)

In [ ]:
bs = 4

In [ ]:
def twoepochs(d): print(' '.join(''.join(o) for _ in range(2) for o in d))

## patch \_MapDatasetFetcher 

In [ ]:
@patch
def fetch(self:torch.utils.data._utils.fetch._MapDatasetFetcher, idxs):
    ds = self.dataset
    if self.auto_collation:
        if hasattr(self.collate_fn,"get_batch"): return self.collate_fn.get_batch(ds, idxs)
        data = [ds[idx] for idx in idxs]
    else: data = ds[idxs]
    return self.collate_fn(data)

In [ ]:
class CustCollate:
    def __init__(self, cf=default_collate): self.cf=cf
    def __call__(self, *args, **kwargs): self.cf(*args, **kwargs)
    def get_batch(self, ds, idxs): return self([ds[idx] for idx in idxs])

In [ ]:
ds = string.ascii_lowercase
dl = DataLoader(ds, batch_size=4, num_workers=0)#, collate_fn=CustCollate())

In [ ]:
[''.join(o) for o in dl]

['abcd', 'efgh', 'ijkl', 'mnop', 'qrst', 'uvwx', 'yz']

## fin